In [559]:
import numpy as np
import networkx as nx
import pylab as plt
from scipy.stats import norm,uniform,binom,poisson
import random
from functools import reduce
from itertools import product
from collections import deque
import itertools
from uuid import uuid4

In [560]:
def float_range(stop, start=0, step=1):
    while start < stop:
        yield float(start)
        start += step 

def tuple_eq(tup1,tup2):
    return len(set(tup1).union(set(tup2))) == len(tup1) == len(tup2)
    
def tuple_rev(tup):
    return (tup[1],tup[0])

def tuple_int(tup1,tup2):
    return len(set(tup1).union(set(tup2))) < len(tup1) + len(tup2)

In [561]:
class Node:
    def __init__(self):
        self._x = None
        self._y = None
        self._weight = None
        self.id = uuid4().hex     
    @property
    def x(self):
        return self._x
    @x.setter
    def x(self, x):
        self._x = x
    @property
    def y(self):
        return self._y
    @y.setter
    def y(self, y):
        self._y = y
    @property
    def weight(self):
        return self._weight
    @weight.setter
    def weight(self, weight):
        self._weight = weight
    @property
    def w(self):
        return self._weight
    @w.setter
    def w(self, w):
        self._weight = w
    @property
    def loc(self):
        return (self.x,self.y)
    @loc.setter
    def loc(self, loc):
        self.x = loc[0]
        self.y = loc[1]
    @property
    def attr(self):
        return {'x':self.x,'y':self.y,'weight':self.weight}
    @attr.setter
    def attr(self, attr):
        for key in attr:
            if key == 'x':
                self.x = attr[key]
            if key == 'y':
                self.y = attr[key]
            if key == 'weight':
                self.weight = attr[key]
    
    def copy(self):
        return Node.make(self.x,self.y,self.weight)
    def __key(self):
        return (self.x,self.y,self.weight,self.id)
    def __hash__(self) -> int:
        return hash(self.__key())  
    def __eq__(self, o: object) -> bool:
        if isinstance(o, Node):
            return self.__key() == o.__key()
        return False 
    def __repr__(self) -> str:
        if self.x == None or self.y == None:
            return f'Node (None,None)'
        string = f'Node ({self.x:.2f},{self.y:.2f})'
        if self.w != None:
            string += f'||weight {self.w:.2f}'
        return string
    def __str__(self) -> str:
        return self.__repr__()
    def __iter__(self):
        return iter(self.loc)
    def __getitem__(self, key):
        return self.loc[key]
    def __abs__(self):
        return np.sqrt(self.x**2 + self.y**2)
    def __contains__(self, item):        
        return item in self.loc
    
    @classmethod
    def make(cls, x, y, weight=None):
        node = cls()
        node.x = x
        node.y = y
        node.weight = weight
        return node
    
    @staticmethod
    def dist(node1, node2):
        return np.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)
    
    def dist_to(self, node):
        return Node.dist(self, node)

In [562]:
class Edge:
    def __init__(self, node1 = None, node2 = None):
        self._node1 = node1 if node1 != None else Node()
        self._node2 = node2 if node2 != None else Node()
        self._weight = None
        self._dir = False
        self.id = uuid4().hex
    @property
    def node1(self):
        return self._node1
    @node1.setter
    def node1(self, node1):
        self._node1 = node1
    @property
    def node2(self):
        return self._node2
    @node2.setter
    def node2(self, node2):
        self._node2 = node2
    @property
    def weight(self):
        return self._weight
    @weight.setter
    def weight(self, weight):
        self._weight = weight
    @property
    def w(self):
        return self._weight
    @w.setter
    def w(self, w):
        self._weight = w
    @property
    def dir(self):
        return self._dir
    @dir.setter
    def dir(self, dir):
        self._dir = dir
    @property
    def nodes(self):
        return (self.node1,self.node2)
    @nodes.setter
    def nodes(self, nodes):
        self.node1 = nodes[0]
        self.node2 = nodes[1]
    @property
    def attr(self):
        return {'node1':self.node1,'node2':self.node2,'weight':self.weight,'dir':self.dir}
    @attr.setter
    def attr(self, attr):
        for key in attr:
            if key == 'node1':
                self.node1 = attr[key]
            if key == 'node2':
                self.node2 = attr[key]
            if key == 'weight':
                self.weight = attr[key]
            if key == 'dir':
                self.dir = attr[key]
    @property
    def node_ids(self):
        return (self.node1.id,self.node2.id)
    @property
    def node_locs(self):
        return (self.node1.loc,self.node2.loc)
    @node_locs.setter
    def node_locs(self, node_locs):
        self.node1.loc = node_locs[0]
        self.node2.loc = node_locs[1]
    @property
    def node_attrs(self):
        return (self.node1.attr,self.node2.attr)
    @node_attrs.setter
    def node_attrs(self, node_attrs):
        self.node1.attr = node_attrs[0]
        self.node2.attr = node_attrs[1]
    @property
    def node_weights(self):
        return (self.node1.weight,self.node2.weight)
    @node_weights.setter
    def node_weights(self, node_weights):
        self.node1.weight = node_weights[0]
        self.node2.weight = node_weights[1]
    @classmethod
    def make(cls, node1, node2, weight=None, dir=False):
        edge = cls(node1, node2)
        edge.weight = weight
        edge.dir = dir
        return edge
    @classmethod
    def make_from_attr(cls, attr):
        edge = cls.make(attr['node1'], attr['node2'], attr['weight'], attr['dir'])
        return edge
    def copy(self):
        return Edge.make_from_attr(self.attr)
    @classmethod
    def reverse(cls, edge):
        return cls.make(edge.node2, edge.node1, edge.weight, edge.dir)
    def reversed(self, keep_id=False):
        if keep_id:
            new_edge = Edge.reverse(self)
            new_edge.id = self.id
            return new_edge
        return Edge.reverse(self)
    def __key(self):
        return (self.node1,self.node2,self.weight,self.dir,self.id)
    def __hash__(self) -> int:
        return hash(self.__key())
    def __eq__(self, o: object) -> bool:
        if isinstance(o, Edge):
            if self.dir:
                return (self.node1,self.node2,self.weight,self.dir,self.id) == (o.node1,o.node2,o.weight,o.dir,o.id)
            else:
                return (self.node1,self.node2,self.weight,self.dir,self.id) == (o.node1,o.node2,o.weight,o.dir,o.id) or (self.node1,self.node2,self.weight,self.dir,self.id) == (o.node2,o.node1,o.weight,o.dir,o.id)
        return False
    def __repr__(self) -> str:
        if self.dir:
            string = f'Edge {self.node1} -> {self.node2}'
        else:
            string = f'Edge {self.node1} -- {self.node2}'
        if self.w != None:
            string += f'||weight {self.w:.2f}'
        return string
    def __str__(self) -> str:
        return self.__repr__()
    def __iter__(self):
        return iter(self.nodes)
    def __getitem__(self, key):
        return self.nodes[key]
    def __abs__(self):
        return self.dist()
    def dist(self):
        return Node.dist(self.node1, self.node2)
    def __contains__(self, item):
        return item in self.nodes
    
    def other_node(self, node):
        if node == self.node1:
            return self.node2
        elif node == self.node2:
            return self.node1
        else:
            raise ValueError('node not in edge')
    def is_loop(self):
        return self.node1 == self.node2
    def is_parallel(self, edge):
        if self.dir:
            return self.node1 == edge.node1 and self.node2 == edge.node2
        else:
            return (self.node1 == edge.node1 and self.node2 == edge.node2) or (self.node1 == edge.node2 and self.node2 == edge.node1)
    def is_reverse(self, edge):
        return self.node1 == edge.node2 and self.node2 == edge.node1
    
    def is_incident(self, node):
        if self.dir:
            return node == self.node2
        return node in self.nodes
    
    
    

In [563]:
class Network:
    def __init__(self, nodes = None, edges = None):
        self.nodes = nodes if nodes else []
        self.edges = edges if edges else []
        self._node_translator = { node.id : node for node in self.nodes }
        self._edge_translator = { edge.id : edge for edge in self.edges }
        self._node_loc = { node.loc : node.id for node in self.nodes }
        self.adj = { node.id : 
                             { 
                              node.id : {
                                         'edge' : [], 'dir_edge' : []
                                        } for node in self.nodes
                             } for node in self.nodes }
        for edge in self.edges:
            self.adj[edge.node1.id][edge.node2.id]['edge'].append(edge.id)
            self.adj[edge.node1.id][edge.node2.id]['dir_edge'].append(edge.id)
            self.adj[edge.node2.id][edge.node1.id]['edge'].append(edge.id)             
                
    def translate_node(self, id):
        return self._node_translator[id]
    def translate_edge(self, id):
        return self._edge_translator[id]
    def get_node(self, loc):
        return self._node_loc[loc]
    def get_edges(self, node1, node2, dir = False):
        if isinstance(node1, Node):
            node1 = node1.id
            node2 = node2.id
        elif isinstance(node1, tuple):
            node1 = self.get_node(node1)
            node2 = self.get_node(node2)
        elif isinstance(node1, str):
            pass
        else:
            raise ValueError('nodes must be type Node, tuple, or str')
        if dir:
            return list(map(self.translate_edge, self.adj[node1][node2]['dir_edge']))
        else:
            return list(map(self.translate_edge, self.adj[node1][node2]['edge']))

        
    @property
    def nodes(self):
        return self._nodes
    @nodes.setter
    def nodes(self, nodes):
        self._nodes = nodes
        self._node_translator = { node.id : node for node in self.nodes }
        self._node_loc = { node.loc : node.id for node in self.nodes }
        if len(self._node_loc) != len(self.nodes):
            raise ValueError('duplicate node locations')
        self.adj = { node.id : 
                             { 
                              node.id : {
                                         'edge' : [], 'dir_edge' : []
                                        } for node in self.nodes
                             } for node in self.nodes }   
        
    @property
    def edges(self):
        return self._edges
    @edges.setter
    def edges(self, edges):
        self._edges = edges
        self._edge_translator = { edge.id : edge for edge in self.edges }
        for edge in self.edges:
            if edge.node1.id not in self.adj or edge.node2.id not in self.adj:
                raise ValueError('edge nodes not in network')
            self.adj[edge.node1.id][edge.node2.id]['edge'].append(edge.id)
            self.adj[edge.node1.id][edge.node2.id]['dir_edge'].append(edge.id)
            self.adj[edge.node2.id][edge.node1.id]['edge'].append(edge.id)
    @edges.deleter
    def edges(self):
        self._edges = []
        self._edge_translator = {}
        for node in self.nodes:
            self.adj[node.id] = { node.id : {
                                         'edge' : [], 'dir_edge' : []
                                        } for node in self.nodes }
    
    def __repr__(self,draw = False) -> str:
        if draw:
            self.draw()
        line_break = '=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-='
        return line_break + '\n' + \
                f"Network with {len(self.nodes)} Nodes and {len(self.edges)} Edges\n" + \
                line_break + '\n' + \
                'Nodes:\n' + '\n'.join([str(node) for node in self.nodes]) + '\n' + \
                line_break + '\n' + \
                'Edges:\n' + '\n'.join([str(edge) for edge in self.edges]) + '\n'
    def __str__(self) -> str:
        return self.__repr__(draw=False)
    def __iter__(self):
        return iter(self.nodes)
    def __getitem__(self, key):
        if isinstance(key, int):
            return self.nodes[key]
        elif isinstance(key, str):
            return self._node_translator[key]
        elif isinstance(key, tuple):
            return self._node_translator[self._node_loc[key]]
        else:
            raise TypeError('key must be int or str')
    def __len__(self):
        return len(self.nodes)
    def __contains__(self, item):
        return item in self.nodes or item in self.edges or item in self._node_translator or item in self._edge_translator
    def __eq__(self, __o: object) -> bool:
        if isinstance(__o, Network):
            self._node_translator == __o._node_translator and self._edge_translator == __o._edge_translator
        else:
            return False
    
    def add_node(self, node):
        if node not in self.nodes:
            if node.loc in self._node_loc:
                raise ValueError('duplicate node location')
            self.nodes.append(node)
            self._node_translator[node.id] = node
            self._node_loc[node.loc] = node.id
            self.adj.update({node.id : { node.id : {
                                         'edge' : [], 'dir_edge' : []
                                        } for node in self.nodes }})
            for node2 in self.nodes:
                self.adj[node2.id].update({node.id : {
                                         'edge' : [], 'dir_edge' : []
                                        }})
        else:
            raise ValueError('node already in network')        
    def add_edge(self, edge):
        if edge.node1 not in self.nodes and edge.node2 not in self.nodes:
            raise ValueError('nodes not in network')
        if edge not in self.edges:
            self.edges.append(edge)
            self._edge_translator[edge.id] = edge
            self.adj[edge.node1.id][edge.node2.id]['edge'].append(edge.id)
            self.adj[edge.node1.id][edge.node2.id]['dir_edge'].append(edge.id)
            self.adj[edge.node2.id][edge.node1.id]['edge'].append(edge.id)        
    def remove_node(self, node):
        if node in self.nodes:
            self.nodes.remove(node)
            del self._node_translator[node.id]
            del self.adj[node.id]
            del self._node_loc[node.loc]
            for node2 in self.nodes:
                del self.adj[node2.id][node.id]
            for edge in self.edges:
                if edge.node1 == node or edge.node2 == node:
                    self.remove_edge(edge)
        else:
            raise ValueError('node not in network')
    def remove_edge(self, edge):
        if edge in self.edges:
            self.edges.remove(edge)
            self.adj[edge.node1.id][edge.node2.id]['edge'].remove(edge.id)
            self.adj[edge.node1.id][edge.node2.id]['dir_edge'].remove(edge.id)
            self.adj[edge.node2.id][edge.node1.id]['edge'].remove(edge.id)
        else:
            raise ValueError('edge not in network')
        
    def add_nodes(self, nodes):
        for node in nodes:
            self.add_node(node)
    def add_edges(self, edges):
        for edge in edges:
            self.add_edge(edge)
    def remove_nodes(self, nodes):
        for node in nodes:
            self.remove_node(node)
    def remove_edges(self, edges):
        for edge in edges:
            self.remove_edge(edge)
    
    def clear(self):
        self.nodes = []
        self.edges = []
        self._node_translator = {}
        self._edge_translator = {}
        self._node_loc = {}
        self.adj = {}
    def copy(self):
        copy_nodes = [node.copy() for node in self.nodes]
        copy_edges = [edge.copy() for edge in self.edges]
        return Network(copy_nodes, copy_edges)
    
    def change_node_weight(self, node, weight):
        try:
            node = self.translate_node(node)
        except:
            pass
        node.weight = weight
    def change_edge_weight(self,edge,weight):
        try:
            edge = self.translate_edge(edge)
        except:
            pass
        edge.weight = weight    
        
    def move_node(self, node, loc ,rel = False):
        try:
            node = self.translate_node(node)
        except:
            pass
        if rel:
            x = node.loc[0] + loc[0]
            y = node.loc[1] + loc[1]
            if (x,y) in self._node_loc:
                raise ValueError('duplicate node location')
            node.loc = (x,y)
        else:
            if loc in self._node_loc:
                raise ValueError('duplicate node location')
            node.loc = loc
            
    def maximal_component(self, node):
        try:
            node = node.id
        except:
            pass    
        visited = set()
        queue = deque([node])
        while queue:
            next = queue.popleft()
            visited.add(next)
            queue.extend([ids for ids in self.adj[next] if ids not in visited and self.adj[next][ids]['edge']])
        return set(map(self.translate_node , visited))
    
    
    
    

In [564]:
a = Node().make(1,2,3)
b = Node().make(1,5,3)
c = Node().make(2,1,4)
d = Node().make(2,5,4)
e = Node().make(1,3,3)
ab = Edge().make(a,b)
ac = Edge().make(a,c)
bd = Edge().make(b,d)
cd = Edge().make(c,d)

In [565]:
nodes = [a,b,c,d,e]
edges = [ab,ac,bd,cd]

In [566]:
network = Network(nodes, edges)

In [567]:
network.add_node(Node().make(2,3,4))

In [568]:
network.get_node((2,3))

'5e206039c8a94ef880d5dd83c254f1da'

In [569]:
network

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Network with 6 Nodes and 4 Edges
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Nodes:
Node (1.00,2.00)||weight 3.00
Node (1.00,5.00)||weight 3.00
Node (2.00,1.00)||weight 4.00
Node (2.00,5.00)||weight 4.00
Node (1.00,3.00)||weight 3.00
Node (2.00,3.00)||weight 4.00
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Edges:
Edge Node (1.00,2.00)||weight 3.00 -- Node (1.00,5.00)||weight 3.00
Edge Node (1.00,2.00)||weight 3.00 -- Node (2.00,1.00)||weight 4.00
Edge Node (1.00,5.00)||weight 3.00 -- Node (2.00,5.00)||weight 4.00
Edge Node (2.00,1.00)||weight 4.00 -- Node (2.00,5.00)||weight 4.00

In [570]:
network.add_edge(Edge().make(a,d))

In [571]:
for i in network.get_edges((1,2),(2,5)):
    print(i)

Edge Node (1.00,2.00)||weight 3.00 -- Node (2.00,5.00)||weight 4.00


In [579]:
network.maximal_component(network.get_node((2,3)))

{Node (2.00,3.00)||weight 4.00}

In [573]:
network._edge_translator

{'a26a8a29f5564cf2b6e60c0aab80c88b': Edge Node (1.00,2.00)||weight 3.00 -- Node (1.00,5.00)||weight 3.00,
 '05e12d9dcac545ce9ef9f614b10aa06b': Edge Node (1.00,2.00)||weight 3.00 -- Node (2.00,1.00)||weight 4.00,
 '6113e91a5b2f4a8a8ae1f4f237de78c6': Edge Node (1.00,5.00)||weight 3.00 -- Node (2.00,5.00)||weight 4.00,
 '3a0b9f08056a40659808f0e591ca063d': Edge Node (2.00,1.00)||weight 4.00 -- Node (2.00,5.00)||weight 4.00,
 'a810db0dd63a456a8c4e1a97ef781347': Edge Node (1.00,2.00)||weight 3.00 -- Node (2.00,5.00)||weight 4.00}

In [574]:
for i in a:
    print(i)

1
2


In [575]:
dic = {1 : {2 : 3, 4 : 5}, 2 : {}}
dic2 = {2 : {},1 : {2 : 3, 4 : 5}}

In [576]:
se = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16}
asd = map(lambda x : x**2, se)
set(asd)

{1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256}

In [577]:
dic2

{2: {}, 1: {2: 3, 4: 5}}

In [578]:
dic == dic2

True